# JalanJalan.ai: Your trusted AI Agent Personal Vacation in Indonesia
“Smart travel, local vibes, powered by AI.”

### by **Danang Agung Restu Aji**

#### Code Reviewer | NLP Research Engineer

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Profdara-blue?logo=linkedin)](https://linkedin.com/in/Profdara)

Dari pulau jawa ke dunia wkwkwkwkw

JalanJalan.ai is an AI-powered travel assistant designed to help explorers discover Indonesia’s hidden gems. From weather-aware destination recommendations to budget planning, JalanJalan.ai coordinates multiple intelligent agents to deliver a seamless vacation plan.

- Location-aware suggestions
- Weather-based destination filtering
- Budget calculation with entrance fees & total cost
- Personalized itineraries for every travel


### Mission
To make traveling in Indonesia effortless, personalized, and unforgettable by blending AI intelligence with local culture.

## agents init
Package initializer for agents.

In [132]:
# Install required packages for this project
import os
from dotenv import load_dotenv

load_dotenv()  # to read file .env , put your api key in .env file please
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    print("API key not found")
else:
    print("✅ API key loaded")



✅ API key loaded


In [133]:
# please pip install if not already installed

# %pip install google-adk

from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [134]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

# main agent

In [135]:
main_agent = Agent(
    name="vacation_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
    ),
    description="A simple agent that supervise other agents to create a complete travel plan.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure, you task is to supervise other agents to create a complete travel plan for a vaautsieon.",
    tools=[google_search],
)
print("✅ main Agent defined.")

✅ main Agent defined.


In [136]:
runner = InMemoryRunner(agent=main_agent)

print("✅ Runner created.")

✅ Runner created.


In [137]:
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
vacation_assistant > The Agent Development Kit (ADK) from Google is a flexible and modular framework designed to simplify the development, deployment, and orchestration of AI agents and multi-agent systems. It aims to make agent development feel more like traditional software development, offering precise control over agent behavior and workflows. The ADK is optimized for Gemini and the Google ecosystem but is also model-agnostic and deployment-agnostic, allowing compatibility with other frameworks.

The ADK is available in the following languages:
*   **Python**
*   **Java**
*   **Go**

Support for additional languages is expected in the future.


In [138]:
# testing weather in Yogyakarta
response = await runner.run_debug("What's the weather in Yogyakarta?")


 ### Continue session: debug_session_id

User > What's the weather in Yogyakarta?
vacation_assistant > The weather in Yogyakarta, Indonesia is currently cloudy with a 0% chance of rain. The temperature is 75°F (24°C) and it feels like 79°F (26°C), with 92% humidity.

Looking ahead, the forecast for Sunday, November 30, 2025, predicts cloudy conditions throughout the day and night, with a 35% chance of rain during the day and a 10% chance at night. Temperatures are expected to range between 75°F (24°C) and 86°F (30°C) with approximately 82% humidity.

Over the next 11 days, Yogyakarta can expect a mix of weather conditions including light rain, thunderstorms, and mostly cloudy skies.


## 1. weather and location agent
WeatherAgent: geocoding (Nominatim) + Open-Meteo forecast.

In [139]:
# weather and location agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="location_weather_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized location and weather agent. Your only job is to use the
    google_search tool to find the nearby weather and location information based on the user's query.""",
    tools=[google_search],
    output_key="location_weather_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ location_weather_agent created.")

✅ location_weather_agent created.


## 2. destination agent


In [140]:
# Summarizer Agent: Its job is to summarize the text it receives.
destination_agent = Agent(
    name="destination_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""
        You are a travel assistant AI.
        Given the user's current location, find nearby tourist attractions (destinations).
        Summarize each destination with:
        - Name (in English)
        - Type (landmark, nature, cultural site, etc.)
        - Distance or estimated travel time
        - One-sentence highlight (why it's interesting)

        Return the summary in a concise, user-friendly format.""",
        tools=[google_search],
    output_key="destination_summary"
)

print("✅ destination_agent created.")

✅ destination_agent created.


## 3. budget agent


In [141]:
# Summarizer Agent: Its job is to summarize the text it receives.
budget_agent = Agent(
    name="budget_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""
    You are a Budget Agent AI.

    Given a list of tourist destinations (with location and weather already considered),
    your tasks are:
    1. Retrieve or estimate the entrance fee for each destination.
    2. Include any additional costs if specified (e.g., transport, meals).
    3. Calculate the total estimated cost for the trip.
    4. Present the result in a clear, user-friendly summary.

    Output must include:
    - Destination name
    - Entrance fee (in local currency)
    - Optional additional costs
    - Total estimated cost

    Return the summary under the key 'budget_summary'.
    """,
    output_key="budget_summary"
)

print("✅ budget_agent created.")

✅ budget_agent created.


## main agent coordinator

In [142]:
main_agent = Agent(
    name="vacation_coordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""
    You are a Vacation Coordinator AI agent.

    Workflow:
    1. Ask the user for their current location.
    2. Send the location to the location_weather_agent to retrieve weather conditions.
    3. Pass both location and weather data to the destination_agent to find suitable tourist attractions.
    - Ensure destinations are filtered based on weather (e.g., avoid outdoor spots in heavy rain).
    4. Forward the selected destinations to the budget_agent.
    - Request entrance fees and calculate the total estimated cost for the trip.
    5. Return a final summary to the user including:
    - Location
    - Weather condition
    - Recommended destinations
    - Estimated entrance fees and total cost

    Output must be structured under the key 'vacation_plan'.
    """,
    output_key="vacation_plan",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(destination_agent), AgentTool(budget_agent)],
)

print("✅ vacation_coordinator_agent created.")

✅ vacation_coordinator_agent created.


In [143]:
runner = InMemoryRunner(agent=main_agent)
response = await runner.run_debug(
    "I'm in Yogyakarta and planning a vacation. Can you help me create a travel plan? my budget is 1 million rupiah"
)


 ### Created new session: debug_session_id

User > I'm in Yogyakarta and planning a vacation. Can you help me create a travel plan? my budget is 1 million rupiah


vacation_coordinator > I've prepared a vacation plan for you in Yogyakarta!

**Location:** Yogyakarta
**Weather Condition:** Cloudy, with a temperature of 24°C (75°F).

**Recommended Destinations:**
*   **Malioboro Street:** A vibrant street for shopping and experiencing local culture. (Free to explore)
*   **Taman Sari (Water Castle):** Explore this historic royal bathing complex. (Entrance fee: ~15,000 - 25,000 IDR)
*   **Sultan's Palace (Kraton Yogyakarta):** Immerse yourself in Javanese culture and history. (Entrance fee: ~15,000 - 25,000 IDR)
*   **Prambanan Temple:** A magnificent UNESCO World Heritage Site, the largest Hindu temple complex in Indonesia. (Domestic entrance fee: ~50,000 IDR)

**Estimated Entrance Fees and Total Cost:**
The total estimated entrance fees for Taman Sari, Sultan's Palace, and Prambanan Temple (domestic rate) come to approximately 80,000 IDR. With your budget of 1,000,000 IDR, you have a remaining budget of 920,000 IDR for other expenses like food, tra